In [3]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pdf2image import convert_from_path
import fitz
import pytesseract
import numpy as np
import re
import os

load_dotenv('../../.env')

True

In [11]:
pdf_path = 'C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf'  

    extract_pdf_data(pdf_path): PDF에서 텍스트 및 이미지 추출
    extract_ocr(images): 이미지에서 OCR을 사용하여 텍스트 추출
    is_text_insufficient(text): 슬라이드의 정보 부족 여부 판단
    compare_text_similarity(text1, text2): 유사도 분석 (TF-IDF 활용)
    detect_tables(image_text): OCR 결과에서 표 감지
    analyze_slide_for_data(slide_text): 데이터 관련 내용 포함 여부 확인
    analyze_image_relevance(image_text, slide_text): OCR 결과와 슬라이드 본문 비교
    generate_presentation_script(slides): 발표 대본 자동 생성
    main(pdf_path): 전체 흐름 실행

In [ ]:
def extract_pdf_data(pdf_path):
    '''PDF에서 텍스트와 이미지를 추출'''
    docs = fitz.open(pdf_path)
    page_data = []

    for doc in docs:
        text = doc.get_text("text").strip()
        images = doc.get_images(full=True)

        image_data = []
        for img in images:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_data.append(image_bytes) 

        page_data.append({"text": text, "images": image_data})

    return page_data

def extract_ocr(images):
    """OCR을 활용하여 이미지에서 텍스트 추출"""
    extracted_texts = []
    
    for image in images:
        text = pytesseract.image_to_string(image, lang="kor+eng")
        extracted_texts.append(text.strip())
    
    return extracted_texts

def is_text_insufficient(text):
    """텍스트 부족 여부 판단"""
    # 기준 1: 텍스트 밀도 분석
    if len(text.split()) < 3:
        return True
    
    # model_name = ""  
    # keywords = [] 
    # if not any(keyword in text for keyword in keywords):
    #     return True
    
    # 기준 2: 문장구조를 분석하는 NLP모델을 사용하여 문장 구조 분석 
    is_valid_sentence = True 
    if not is_valid_sentence:
        return True
    
    return False

def compare_text_similarity(text1, text2):
    """텍스트 유사도 분석"""
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    similarity = cosine_similarity([vectors[0]], [vectors[1]])[0][0]
    return similarity

def detect_tables(image_text):
    """OCR 결과에서 표를 감지 (단순 행렬 구조 확인)"""
    lines = image_text.split("\n")
    table_like_lines = [line for line in lines if re.search(r'\d+[\s|,|\t]+\d+', line)] 
    return len(table_like_lines) > 2  # 일정 개수 이상의 행이 감지되면 표로 간주

def analyze_slide_for_data(slide_text):
    """슬라이드에서 데이터 관련 용어가 포함되었는지 확인"""
    data_keywords = ["통계", "데이터", "비율", "퍼센트", "%", "결과", "수치", "값", "평균", "표"]
    return any(keyword in slide_text for keyword in data_keywords)

def analyze_image_relevance(image_text, slide_text):
    """이미지 내 텍스트(OCR 결과)와 슬라이드 본문 비교"""
    similarity = compare_text_similarity(image_text, slide_text)
    return similarity >= 0.8  # 유사도가 80% 이상이면 관련 있음으로 판단

def detect_image_caption(image, slide_text):
    """이미지 설명 문구(캡션) 감지 및 연관성 평가"""
    pass

def analyze_table_in_slide(image_text, slide_text):
    """표와 슬라이드 데이터 관련성 분석"""
    if detect_tables(image_text) and analyze_slide_for_data(slide_text):
        return True 
    return False

def analyze_slide_text(text, next_slide_text):
    """현재 슬라이드의 내용이 부족할 경우 후속 슬라이드 참조"""
    if is_text_insufficient(text):
        return f"현재 슬라이드의 정보가 부족하여 다음 슬라이드 내용을 참고합니다: {next_slide_text}"
    return text

def generate_presentation_script(slides):
    """발표 대본 생성 (후속 슬라이드 참조)"""
    pass

def main(pdf_path):
    """전체 흐름 제어"""
    pass



In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/t5-v1_1-base"  # 사용할 모델 선택
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 로컬에 저장
model.save_pretrained("../../model/t5_model")
tokenizer.save_pretrained("../../model/t5_model")
# 더 가벼운거
# tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
# model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

c:\wanted\Lang\lang\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\wanted\Lang\lang\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--google--t5-v1_1-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https

('../../model/t5_model\\tokenizer_config.json',
 '../../model/t5_model\\special_tokens_map.json',
 '../../model/t5_model\\spiece.model',
 '../../model/t5_model\\added_tokens.json')

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
# device = "cuda" if torch.cuda.is_available() else "cpu"
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)
# timeout=60 (기본 10초 → 60초로 증가)
model_name = "Salesforce/blip2-opt-2.7b"
processor = Blip2Processor.from_pretrained(model_name, timeout=60)
model = Blip2ForConditionalGeneration.from_pretrained(model_name).to("cuda")

# 로컬에 저장
model.save_pretrained("../../model/blip2")
tokenizer.save_pretrained("../../model/blip2")

In [9]:
from transformers import BlipProcessor, BlipForConditionalGeneration

model_name = "Salesforce/blip-image-captioning-base"  # 사용할 BLIP 모델 선택
save_path = "../../model/blip_model"  # 저장할 폴더 지정

# 모델 다운로드 및 로컬 저장
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

processor.save_pretrained(save_path)
model.save_pretrained(save_path)

print(f"BLIP 모델이 {save_path}에 저장되었습니다.")


BLIP 모델이 ../../model/blip_model에 저장되었습니다.


In [ ]:
import fitz  # PyMuPDF
import easyocr
import paddleocr
import torch
import json
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from PIL import Image
from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer, util
import io
from deep_translator import GoogleTranslator

# 모델 경로를 JSON 파일에서 로드
def load_model_paths(config_path="model_config.json"):
    with open(config_path, "r") as file:
        config = json.load(file)
    return config["t5_model_path"], config["blip_model_path"]

# 사전 학습된 모델을 로컬에 저장 후 로드
def load_t5_model(model_path):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    text_generation_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
    return tokenizer, model, text_generation_pipeline

def load_blip_model(model_path):
    processor = BlipProcessor.from_pretrained(model_path)
    model = BlipForConditionalGeneration.from_pretrained(model_path)
    return processor, model

# PDF에서 페이지별 텍스트 및 이미지 추출
def extract_content_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    pages_content = []
    
    for page_num in range(len(doc)):
        # 텍스트 추출
        text = doc[page_num].get_text("text").strip()
        if not text:
            text = "[페이지에 텍스트 없음]"
        
        # 이미지 추출
        images = []
        for img_index, img in enumerate(doc[page_num].get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            images.append(image)
        
        # 페이지별 텍스트 및 이미지 저장
        pages_content.append({"text": text, "images": images})
    
    return pages_content

# OCR을 사용하여 이미지에서 텍스트 추출
def extract_text_from_image(image, use_easyocr=True):
    image_np = np.array(image)
    ocr_reader = easyocr.Reader(['en']) if use_easyocr else paddleocr.OCR()
    extracted_text = ""
    
    if use_easyocr:
        results = ocr_reader.readtext(image_np)
        extracted_text = " ".join([text[1] for text in results])
    else:
        results = ocr_reader.ocr(image_np)
        extracted_text = " ".join([text[1][0] for text in results])
    
    if not extracted_text.strip():
        extracted_text = "[OCR 인식 불가]"
    
    return extracted_text

# BLIP-2를 사용하여 이미지 설명 생성 및 번역
def generate_caption(image, blip_processor, blip_model):
    image = image.convert("RGB")
    inputs = blip_processor(images=image, return_tensors="pt")
    caption_ids = blip_model.generate(**inputs)
    caption = blip_processor.decode(caption_ids[0], skip_special_tokens=True)
    
    if caption.count("black background") > 2:
        caption = "[이미지 설명 불가능]"
    else:
        caption = GoogleTranslator(source='en', target='ko').translate(caption)
    
    return caption

# 문장 유사도 판별
def is_text_similar(text1, text2, threshold=0.7):
    model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
    return similarity >= threshold, similarity

# T5 모델을 LangChain을 사용하여 발표 대본 생성
def generate_script(text, text_generation_pipeline):
    if len(text.strip()) < 10:
        return "[대본 생성 불가: 입력 데이터 부족]"
    
    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""
        당신은 전문 발표자입니다. 주어진 내용을 기반으로 청중이 이해하기 쉬운 발표 대본을 작성하세요.
        다음은 슬라이드의 내용입니다:
        {text}
        
        이에 대한 발표 대본을 생성하세요.
        """
    )
    
    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
    chain = LLMChain(llm=llm, prompt=prompt_template)
    response = chain.run(text=text)
    
    return response

# 전체 발표 대본 생성 과정
def generate_presentation_script(pdf_path, config_path="model_config.json"):
    t5_model_path, blip_model_path = load_model_paths(config_path)
    pages_content = extract_content_from_pdf(pdf_path)
    tokenizer, t5_model, text_generation_pipeline = load_t5_model(t5_model_path)
    blip_processor, blip_model = load_blip_model(blip_model_path)
    
    scripts = []
    
    for idx, page in enumerate(pages_content):
        text = page["text"]
        ocr_text = ""
        caption_text = ""
        
        for image in page["images"]:
            ocr_text += extract_text_from_image(image) + " "
            caption = generate_caption(image, blip_processor, blip_model)
            is_similar, similarity = is_text_similar(text, caption)
            if is_similar:
                caption_text += caption + " "
        
        combined_text = f"{text} {caption_text}".strip()
        script = generate_script(combined_text, text_generation_pipeline)
        scripts.append(f"슬라이드 {idx+1} 발표 대본: {script}")
    
    return scripts

# PDF 파일 경로
pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"
config_path = "model_config.json"

# 발표 대본 생성 실행
scripts = generate_presentation_script(pdf_path, config_path)

# 결과 출력
for i, script in enumerate(scripts):
    print(f"슬라이드 {i+1} 발표 대본:\n{script}\n")

Device set to use cuda:0


In [ ]:
import fitz  # PyMuPDF
import easyocr
import paddleocr
import torch
import json
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from PIL import Image
from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer, util
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import io
from deep_translator import GoogleTranslator
from langchain.memory import ConversationBufferMemory    # 메모리 부여
from langchain.chains import ConversationChain  # 메모리 부여

In [ ]:
def extract_content_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    pages_content = []
    
    for page_num in range(len(doc)):
        # 텍스트 추출
        text = doc[page_num].get_text("text").strip()
        if not text:
            text = "[페이지에 텍스트 없음]"
        
        # 이미지 추출
        images = []
        for img_index, img in enumerate(doc[page_num].get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            images.append(image)
        
        # 페이지별 텍스트 및 이미지 저장
        pages_content.append({"text": text, "images": images})
    return pages_content

# OCR을 사용하여 이미지에서 텍스트 추출
def extract_text_from_image(image, use_easyocr=True):
    image_np = np.array(image)
    ocr_reader = easyocr.Reader(['en']) if use_easyocr else paddleocr.OCR()
    extracted_text = ""
    
    if use_easyocr:
        results = ocr_reader.readtext(image_np)
        extracted_text = " ".join([text[1] for text in results])
    else:
        results = ocr_reader.ocr(image_np)
        extracted_text = " ".join([text[1][0] for text in results])
    
    if not extracted_text.strip():
        extracted_text = "[OCR 인식 불가]"
    
    return extracted_text


def generate_caption(image, blip_processor, blip_model):
    image = image.convert("RGB")
    inputs = blip_processor(images=image, return_tensors="pt")
    caption_ids = blip_model.generate(**inputs)
    caption = blip_processor.decode(caption_ids[0], skip_special_tokens=True)
    
    if caption.count("black background") > 2:
        caption = "[이미지 설명 불가능]"
    else:
        caption = GoogleTranslator(source='en', target='ko').translate(caption)
    
    return caption

def is_text_similar(text1, text2, threshold=0.7):
    model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
    return similarity >= threshold, similarity

def generate_script(text):

    llm = ChatOllama(model='mistral:7b', temperature=0.4)

    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""
        당신은 전문 발표자입니다. 
        주어진 내용을 기반으로 청중이 이해하기 쉬운 발표 대본을 작성하세요.
        발표 대본은 문단이 하나입니다.
        한글로만 말하세요.
        다음은 슬라이드의 내용입니다:
        {text}
        
        이에 대한 발표 대본을 생성하세요.
        """
    )
    memory = ConversationBufferMemory()

    chain = ConversationChain()

    result = chain.invoke({'text' : text})
    return result

memory = ConversationBufferMemory()

con_chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [46]:
pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"
ppt_content = extract_content_from_pdf(pdf_path)
image = ppt_content[20]['images']

In [47]:
# image[0].show()
# len(image)
for i in image:
    i.show()

In [ ]:
# texts
processor = BlipProcessor.from_pretrained("../../model/blip_model")
model = BlipForConditionalGeneration.from_pretrained("../../model/blip_model")

for page, content in enumerate(ppt_content):
    for image in content['images']:
        if extract_text_from_image(image) != "[OCR 인식 불가]":
            print(f"page : {page} , text : {content['text']}\nOCR : {extract_text_from_image(image)}, 해석 : {generate_caption(image, processor, model)}")
            relation = is_text_similar(
                content['text'],
                generate_caption(image, processor, model) + extract_text_from_image(image)
            )
            print(f"ppt에 필요한 이미지인가? : {relation}")
        print(F"대본 \n {generate_script(content['text']).content}")
        print("=" * 120)

page : 2 , text : HTP 검사란?
주제 소개
House(집), Tree(나무), Person(사람)의 세
가지 그림을 그리게 하여 개인의 성격, 정서 상
태, 대인 관계 등을 평가하는 투사적 심리 검사
특히 아동 및 청소년의 심리 상태를 파악하거나,
성인의 무의식적 감정과 스트레스를 이해하는
데 유용하게 사용
01. 프로젝트 개요
OCR : HTP HTP, 해석 : 나무와 연필이있는 집의 그림
ppt에 필요한 이미지인가? : (False, 0.41756924986839294)
대본 
  "안녕하세요, 모두! 오늘은 우리는 'HTP 검사'라는 시각적인 심리적 평가 도구에 대해 알아보도록 합니다. 슬라이드에 나와 있듯이, 이 검사는 '집', '나무', '사람' 세 그림을 그리게 하여 개인의 성격, 정서 상태, 대인관계 등을 평가합니다. 특히 아동과 청소년의 심리 상황에 유용하며, 성인들이 무의식적 감정과 스트레스를 이해하는데 도움이 되기도 합니다. 오늘은 'HTP 검사' 프로젝트에 대한 개요를 알아보겠습니다. 감상하시고 이해하시면 됩니다! 지금부터!"
page : 3 , text : HTP 검사의 한계
주제 선정 배경
주관적 해석의 가능성
표준화 부족
문화적 차이
피검사자의 의도적 왜곡
기술적 한계
01. 프로젝트 개요
OCR : DRAWING TEST HTP ORCEZOCi gntON PERSON TREE HOUSE, 해석 : 그 위에 그림이있는 책상에 앉아있는 남자
ppt에 필요한 이미지인가? : (False, 0.3413795232772827)
대본 
  Title Slide: Limitations of HTP Testing

   Welcome everyone, today I will discuss the limitations of the Homeostatic Thyroid Prophyruron (HTP) test, a commonly used diagnostic tool in thyroid function assessment. Despite

In [ ]:
import pdfplumber
import easyocr
import torch
import os
import io
import cv2
import numpy as np
from langchain.document_loaders import PDFPlumberLoader
from transformers import BlipProcessor, BlipForConditionalGeneration
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PIL import Image

# ✅ PDF 경로
pdf_path = "../../data/pdf/presentation_agent.pdf"

# ✅ OCR 모델 (EasyOCR)
ocr_reader = easyocr.Reader(["en", "ko"])  # 한국어 + 영어 지원

# ✅ BLIP-2 모델 (이미지 설명 AI)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 1번 GPU만 사용
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

# ✅ OpenCV를 활용하여 PIL 이미지를 NumPy 배열로 변환
def image_to_numpy(image_pil):
    """PIL 이미지를 NumPy 배열로 변환 (EasyOCR 입력용)"""
    image_cv = np.array(image_pil.convert("RGB"))  # RGB 변환
    image_gray = cv2.cvtColor(image_cv, cv2.COLOR_RGB2GRAY)  # ✅ 흑백 변환 (OCR 성능 향상)
    return image_gray  # ✅ EasyOCR는 grayscale을 선호함

# ✅ PDF 분석 및 페이지 분류 + 이미지 처리
def extract_page(pdf_path):
    loader = PDFPlumberLoader(pdf_path)
    documents = loader.load()
    pages = []
    total_pages = len(documents)

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, doc_page in enumerate(documents):
            page = pdf.pages[page_num]
            text = doc_page.page_content.strip() if doc_page.page_content else ""

            # ✅ 이미지 처리
            images = []
            for img in page.images:
                x0, y0, x1, y1 = img["x0"], img["top"], img["x1"], img["bottom"]
                xyxy = x0, y0, x1, y1
                images.append(xyxy)
                # ✅ 이미지 크롭 후 OCR 적용
                page_image = page.to_image()
                full_img = page_image.annotated
                img_crop = full_img.crop((x0, y0, x1, y1))
                img_bytes = io.BytesIO()
                img_crop.save(img_bytes, format="PNG")
                img_pil = Image.open(img_bytes)

                # # ✅ OCR 적용 (EasyOCR)
                # ocr_input = image_to_numpy(img_pil)  # OpenCV 변환 후 OCR 실행
                # ocr_text = ocr_reader.readtext(ocr_input, detail=0)
                # ocr_result = " ".join(ocr_text) if ocr_text else "No OCR text found."

                # ✅ 이미지 설명 (BLIP-2)
                img_pil = img_pil.convert("RGB")  # BLIP-2에서 RGB 변환 필수
                inputs = processor(img_pil, return_tensors="pt").to(device)
                with torch.no_grad():
                    caption = model.generate(**inputs)
                    blip_caption = processor.decode(caption[0], skip_special_tokens=True)

                images.append({
                    "image_id": len(images) + 1,
                    "ocr_text": xyxy,
                    "blip_caption": blip_caption
                })

            pages.append({
                "page": page_num + 1,
                "text": text,
                "images": images
            })

    return pages

In [59]:
pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"
page_info = extract_page(pdf_path)

In [60]:
page_info[0]

{'page': 1,
 'text': 'Team D.P. Wanted PotenUp\n3rd Project\n딥러닝 기반 아동 미술 심리 진단\nDeepPrint\n김지민, 박형빈, 정재식\n2025.03.05.',
 'images': [(-422.643150389868,
   -227.99541783018992,
   1440.356771985132,
   1013.254650451055),
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1863x1241>,
  {'image_id': 3,
   'ocr_text': (-422.643150389868,
    -227.99541783018992,
    1440.356771985132,
    1013.254650451055),
   'blip_caption': 'there is a blackboard with a drawing of a bird and a flower',
   'image_pil': <PIL.Image.Image image mode=RGB size=1863x1241>}]}

In [62]:
page_info[0]['images'][2]['image_pil'].show()